In [1]:
!pip install split-folders
from keras.applications import ResNet101, DenseNet121
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras import layers
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D, BatchNormalization, Activation, Add, Lambda, GlobalMaxPooling2D, Concatenate, Dropout, Flatten

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import splitfolders  # or import split_folders
import tensorflow as tf
from keras.layers import Input, Multiply
from tensorflow.keras import mixed_precision
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.layers import SeparableConv1D

2024-10-10 23:28:00.790071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 23:28:00.790157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 23:28:00.792012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
splitfolders.ratio('/kaggle/input/datasetss/tomato_state', output="output", seed=1337, ratio=(.8, .2), group_prefix=None) # default values





Copying files: 3196 files [00:03, 923.21 files/s] 


In [3]:
print("Training dataset :")# Définir le générateur d'images pour l'entraînement
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
#                                  rotation_range=40,
#                                  width_shift_range=0.2,
#                                  height_shift_range=0.2,
#                                  shear_range=0.2,
#                                  zoom_range=0.2,
#                                  horizontal_flip=True,
#                                  vertical_flip=True,
#                                  featurewise_std_normalization=True,
#                                  fill_mode='nearest'      # Remplissage des pixels vides après transformation
)

# Créer le dataset d'entraînement
train_dataset = train_datagen.flow_from_directory('./output/train',
                                                  target_size=(256,256),
                                                  color_mode='rgb',
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                  shuffle=True)

Training dataset :
Found 2556 images belonging to 2 classes.


In [4]:
print("Validation dataset :")
# Générateur d'images pour la validation (sans augmentation)
val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
#                                 rotation_range=40,
#                                 width_shift_range=0.2,
#                                 height_shift_range=2.2,
#                                 shear_range=0.2,
#                                 zoom_range=0.2,
#                                 horizontal_flip=True,
#                                 vertical_flip=True,
#                                 featurewise_std_normalization=True,
#                                 fill_mode='nearest'
                              ) 



# Créer le dataset de validation
val_dataset = val_datagen.flow_from_directory('./output/val',
                                              target_size=(256,256),
                                              color_mode='rgb',
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=True
)

Validation dataset :
Found 640 images belonging to 2 classes.


In [5]:
def ssd_extractor(x):
    conv6 = layers.SeparableConv2D(1024, (2, 2), padding='same', activation='relu', name='conv6')(x)
    conv7 = layers.SeparableConv2D(1024, (2, 2), padding='same', activation='relu', name='conv7')(conv6)

    feature_layers = [conv7]
    for filters, name in [(512, 'conv8_2'), (512, 'conv9_2')]:
        feature_layers.append(tf.keras.layers.Conv2D(filters, (1, 1), activation='relu', padding='same', name=name)(feature_layers[-1]))
        features = layers.Concatenate(name='ssd_feature_concat')(feature_layers)

    
    return features
    
# Set mixed precision policy
mixed_precision.set_global_policy('mixed_float16')

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, SeparableConv2D, Conv2D, Layer
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model

class ECALayer(Layer):
    def __init__(self, **kwargs):
        super(ECALayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        pass  # Add your custom layer implementation here if needed
    
    def call(self, inputs):
        return inputs  # Modify based on actual ECA functionality


In [13]:
from keras.applications.resnet import ResNet101, preprocess_input
from keras.layers import Input, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten
from keras.models import Model
from keras.regularizers import l2

input_shape = (256, 256, 3)
inputs = Input(shape=input_shape)
preprocessed_input = preprocess_input(inputs)  # Use ResNet101 preprocessing

# Charger et prétraiter le modèle ResNet101
base_model = ResNet101(include_top=False, weights='imagenet', input_shape=input_shape)
base_model.trainable = False  # Geler les couches du modèle de base

# Débloquer certaines couches supérieures (optionnel)
# for layer in base_model.layers[-10:]:
#     layer.trainable = True

# Passer les entrées à travers le modèle de base
x = base_model(preprocessed_input)
# x = ECALayer(x)
# x = ssd_extractor(x)
# Ajouter des couches de classification
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.2)(x)
# # x = BatchNormalization()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.2)(x)
# x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
# x = BatchNormalization()(x)
x = Dropout(0.2)(x)
# x = Flatten()(x)
outputs = Dense(2, activation='softmax')(x)  # Pour une classification binaire

# Créer le modèle complet
model = Model(inputs=inputs, outputs=outputs)

In [14]:
# for layer in model.layers[-10:]:
#     layer.trainable=True
# for layer in model.layers[20:]:
#     layer.trainable=True

In [15]:
# Afficher la structure du modèle complet
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_6          │ (None, 256, 256)  │          0 │ input_layer_4[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_7          │ (None, 256, 256)  │          0 │ input_layer_4[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_8          │ (None, 256, 256)  │          0 │ input_layer_4[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack_2 (Stack)     │ (None, 256, 256,  │          0 │ get_item_6[0][0], │
│                     │ 3)                │            │ get_item_7[0][0], │
│                     │                   │            │ get_item_8[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 256, 256,  │          0 │ stack_2[0][0]     │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet101           │ (None, 8, 8,      │ 42,658,176 │ add_2[0][0]       │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet101[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1024)      │  2,098,176 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 1024)      │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1024)      │  1,049,600 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 1024)      │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 512)       │    524,800 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 512)       │          0 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 2)         │      1,026 │ dropout_8[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 46,331,778 (176.74 MB)

 Trainable params: 3,673,602 (14.01 MB)

 Non-trainable params: 42,658,176 (162.73 MB)

In [16]:
 optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)  # for clipping gradients by norm

# # Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
# Réduire le taux d'apprentissage si la validation loss stagne
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_lr=1e-5)
# Entraîner le modèle avec les données d'entraînement et de validation
my_model = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[reduce_lr]
)


Epoch 1/10


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 2), output.shape=(None, 2)

In [ ]:
test_loss, test_accuracy = model.evaluate(val_dataset)  # Utilisez val_dataset ou test_dataset si vous en avez un

In [ ]:
import matplotlib.pyplot as plt

# Extraction des données d'accuracy et de loss
train_accuracy = my_model.history['accuracy']
val_accuracy = my_model.history['val_accuracy']
train_loss = my_model.history['loss']
val_loss = my_model.history['val_loss']

epochs = range(1, len(train_accuracy) + 1)

# Plot de l'accuracy
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.legend()

# Plot de la loss
plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
### from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img


class_names = list(train_dataset.class_indices.keys())
#print("Class Names:", class_names)
#print("-" * 40)
img_path = '/kaggle/input/intern/final_image_with_black_background.png'
image = cv2.imread(img_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
plt.axis('off')  # Masquer les axes
plt.show()
img = load_img(img_path, target_size=(256, 256))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create a batch
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions[0])
predicted_class_name = class_names[predicted_class_index]
#print(predictions)
#print("-" * 40)
#print('Predicted class index:', predicted_class_index)
#print("-" * 40)
print('Predicted class name:', predicted_class_name)
#print("-" * 40)
if predicted_class_name == 'Tomato___healthy':
    print('plant is healthy, no need to do treatment')
else:
    print('plant is diseased , it needs treatment')
print("-" * 40)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Make predictions
# Predict class probabilities for the validation set
val_predictions = model.predict(val_dataset)

# Step 2: Convert predictions to class labels
# Get the class with the highest probability for each prediction
val_predicted_classes = np.argmax(val_predictions, axis=1)

# Step 3: Get true labels from the validation generator
# Since val_generator.classes gives the true labels of the validation data
val_true_classes = val_dataset.classes

# Step 4: Generate the confusion matrix
conf_matrix = confusion_matrix(val_true_classes, val_predicted_classes)

# Step 5: Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=val_dataset.class_indices.keys(), yticklabels=val_dataset.class_indices.keys())
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.title('Matrice de confusion M3: detection ')
plt.show()